<a href="https://colab.research.google.com/github/Vinizx17/ArangoDB-NLP/blob/main/Arango_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="text-align: center;">
    <img src="https://github.com/arangodb/interactive_tutorials/blob/master/notebooks/img/ArangoDB_logo.png?raw=1" width="300"/>
</div>

## Instalação de Dependências

Nesta seção, instalamos todos os pacotes Python necessários para o tutorial. Inclui bibliotecas para manipulação de dados (`pandas`), interação com ArangoDB (`python-arango`, `pyarango`), modelos de NLP (`spacy`) e modelos de geração de texto (`transformers`).

Também baixamos o modelo de português `pt_core_news_sm` do spaCy para análise de linguagem natural.

Por fim, clonamos o repositório de tutoriais da ArangoDB para acessar exemplos prontos.


In [64]:
%%capture

!pip install pandas python-arango pyarango spacy

!python -m spacy download pt_core_news_sm

# Clone repo
!git clone https://github.com/arangodb/interactive_tutorials.git -b oasis_connector --single-branch
!rsync -av interactive_tutorials/ ./ --exclude=.git


In [65]:
import pandas as pd
from arango import ArangoClient
import oasis
import zipfile
import requests
import spacy


## Conexão com ArangoDB

Aqui conectamos ao banco ArangoDB Oasis.

- Recuperamos credenciais temporárias fornecidas pelo serviço de tutoriais (`oasis.getTempCredentials`).  
- Conectamos usando `oasis.connect_python_arango`.  
- Mostramos na tela o host, porta, usuário e senha temporários.  

Essa conexão será usada em todas as operações de leitura e escrita do tutorial.


In [71]:
# Retrieve tmp credentials from ArangoDB Tutorial Service
login = oasis.getTempCredentials(tutorialName="Movielens", credentialProvider='https://tutorials.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB')

# Connect to the temp database
db = oasis.connect_python_arango(login)

print("https://{}:{}".format(login["hostname"], login["port"]))
print("Username: " + login["username"])
print("Password: " + login["password"])
print("Database: " + login["dbName"])

Reusing cached credentials.
https://tutorials.arangodb.cloud:8529
Username: TUTyldl39um9rc84y6lg24jqi
Password: TUTznnfzlfk4j23pckee6gb
Database: TUTqhru6uan385a0q35wthbb


## Inserção dos dados

Nesta seção carregamos e inserimos os dados no ArangoDB.

1. Baixamos um CSV de clientes de um repositório público do GitHub.  
2. Conectamos novamente ao ArangoDB usando dados fixos (opcional se quiser usar suas próprias credenciais).  
3. Criamos as coleções e o grafo (`Customers`, `Stores`, `Cities`, `LIVES_IN`, `PREFERRED_STORE`) se não existirem.  
4. Referenciamos essas coleções para facilitar operações futuras.  
5. Preparamos os dados para **bulk insert**, separando clientes, cidades, lojas e as arestas do grafo.  
6. Inserimos os dados em massa no banco para criar o grafo de clientes, cidades e lojas.

In [ ]:
%%capture

# -------------------------------------------------------
# 1. Baixar CSV do GitHub
url = "https://raw.githubusercontent.com/microsoft/fabric-samples/main/community-samples/PII%20SparkShield/data/customer-profile-sample-data/customers-sampledata.csv"
df = pd.read_csv(url)
print("Amostra do CSV:")
print(df.head())


In [67]:
# -------------------------------------------------------
# 2. Conexão com ArangoDB Oasis
# Fixed connection data (replace with your own)
HOST = "https://tutorials.arangodb.cloud:8529"
DB_NAME = "TUTqhru6uan385a0q35wthbb" #Your DB name
USERNAME = "TUTyldl39um9rc84y6lg24jqi" #Your Username
PASSWORD = "TUTznnfzlfk4j23pckee6gb" #Your Password

# Set up connection
client = ArangoClient(hosts=HOST)
db = client.db(DB_NAME, username=USERNAME, password=PASSWORD)


# -------------------------------------------------------
# 3. Criar coleções e grafo (se não existirem)
for col_name in ["Customers", "Stores", "Cities", "LIVES_IN", "PREFERRED_STORE"]:
    if not db.has_collection(col_name):
        db.create_collection(col_name, edge=(col_name in ["LIVES_IN", "PREFERRED_STORE"]))

if not db.has_graph("CustomerGraph"):
    db.create_graph(
        "CustomerGraph",
        edge_definitions=[
            {
                "edge_collection": "LIVES_IN",
                "from_vertex_collections": ["Customers"],
                "to_vertex_collections": ["Cities"]
            },
            {
                "edge_collection": "PREFERRED_STORE",
                "from_vertex_collections": ["Customers"],
                "to_vertex_collections": ["Stores"]
            }
        ]
    )

# -------------------------------------------------------
# 4. Referência às coleções
customers = db.collection("Customers")
stores = db.collection("Stores")
cities = db.collection("Cities")
lives_in = db.collection("LIVES_IN")
preferred_store = db.collection("PREFERRED_STORE")

# -------------------------------------------------------
# 5. Preparar dados para bulk insert
customers_docs = []
cities_docs = {}
stores_docs = {}
edges_lives_in = []
edges_preferred_store = []

for _, row in df.iterrows():
    cust_key = row["CustomerID"]

    # Cliente
    customers_docs.append({
        "_key": cust_key,
        "FirstName": row["FirstName"],
        "LastName": row["LastName"],
        "Email": row["Email"],
        "PhoneNumber": row["PhoneNumber"],
        "DateOfBirth": row["DateOfBirth"],
        "Gender": row["Gender"],
        "RegistrationDate": row["RegistrationDate"],
        "LoyaltyPoints": row["LoyaltyPoints"],
        "MarketingOptIn": row["MarketingOptIn"]
    })

    # Cidade
    city_key = row["City"].replace(" ", "_") + "_" + row["State"]
    if city_key not in cities_docs:
        cities_docs[city_key] = {
            "_key": city_key,
            "City": row["City"],
            "State": row["State"],
            "Country": row["Country"],
            "PostalCode": row["PostalCode"]
        }

    # Loja
    store_key = row["PreferredStore"].replace(" ", "_")
    if store_key not in stores_docs:
        stores_docs[store_key] = {
            "_key": store_key,
            "StoreName": row["PreferredStore"]
        }

    # Arestas
    edges_lives_in.append({
        "_from": f"Customers/{cust_key}",
        "_to": f"Cities/{city_key}"
    })
    edges_preferred_store.append({
        "_from": f"Customers/{cust_key}",
        "_to": f"Stores/{store_key}"
    })

# -------------------------------------------------------
# 6. Bulk insert
customers.import_bulk(customers_docs, overwrite=True)
cities.import_bulk(list(cities_docs.values()), overwrite=True)
stores.import_bulk(list(stores_docs.values()), overwrite=True)
lives_in.import_bulk(edges_lives_in, overwrite=True)
preferred_store.import_bulk(edges_preferred_store, overwrite=True)


{'error': False,
 'created': 20,
 'errors': 0,
 'empty': 0,
 'updated': 0,
 'ignored': 0,
 'details': []}

## Prompt com exemplos de perguntas


Aqui criamos funções para responder perguntas em linguagem natural sobre os clientes.  

- `extract_intent_entities(question)`: identifica intenção da pergunta e extrai entidades (cidade, loja, gênero, pontos de fidelidade, data de cadastro).  
- `build_aql(intent, entities)`: constrói consultas AQL com base na intenção e entidades identificadas.  
- `answer_question(question)`: função principal que executa a AQL no ArangoDB e retorna um resumo de clientes correspondente à pergunta.

Essa seção permite consultar o grafo usando perguntas do tipo:  
- "Quais clientes moram na cidade de X?"  
- "Quem prefere a loja Y?"  
- "Quais clientes têm pontos de fidelidade acima de 500?"

In [69]:
# Inicializa spaCy
nlp = spacy.load("pt_core_news_sm")  # modelo em português

# Função para identificar intenção e entidades
def extract_intent_entities(question):
    doc = nlp(question.lower())
    intent = None
    entities = {}

    # Identificar cidade (entidade GPE)
    for ent in doc.ents:
        if ent.label_ in ["LOC", "GPE"]:
            entities['city'] = ent.text.title()

    # Palavras-chave de intenção
    if any(token.text in ["cidade", "mora", "reside"] for token in doc):
        intent = "city"
    elif any(token.text in ["loja", "prefere", "frequenta"] for token in doc):
        intent = "store"
        # tenta extrair nome da loja do texto
        entities['store'] = question.split("loja")[-1].strip().replace("?", "").title()
    elif any(token.text in ["gênero", "sexo"] for token in doc):
        intent = "gender"
        entities['gender'] = question.split("gênero")[-1].strip().replace("?", "").title()
    elif "pontos de fidelidade" in question:
        intent = "loyalty_points"
        import re
        match = re.search(r"acima de (\d+)", question)
        if match:
            entities['points'] = int(match.group(1))
    elif "cadastro" in question and "após" in question:
        intent = "registration_date"
        import re
        match = re.search(r"após (\d{4}-\d{2}-\d{2})", question)
        if match:
            entities['date'] = match.group(1)

    return intent, entities

# Função para gerar AQL
def build_aql(intent, entities):
    if intent == "city" and 'city' in entities:
        city_name = entities['city']
        return f"""
        WITH Customers, Cities, LIVES_IN
        FOR c IN Customers
            FOR ci IN OUTBOUND c LIVES_IN
                FILTER ci.City == "{city_name}"
                RETURN {{
                    CustomerID: c._key,
                    Nome: CONCAT(c.FirstName, " ", c.LastName),
                    Email: c.Email,
                    Cidade: ci.City
                }}
        """
    elif intent == "store" and 'store' in entities:
        store_name = entities['store']
        return f"""
        WITH Customers, Stores, PREFERRED_STORE
        FOR c IN Customers
            FOR s IN OUTBOUND c PREFERRED_STORE
                FILTER s.StoreName == "{store_name}"
                RETURN {{
                    CustomerID: c._key,
                    Nome: CONCAT(c.FirstName, " ", c.LastName),
                    Email: c.Email,
                    Loja: s.StoreName
                }}
        """
    elif intent == "gender" and 'gender' in entities:
        gender = entities['gender']
        return f"""
        FOR c IN Customers
            FILTER c.Gender == "{gender}"
            RETURN {{
                CustomerID: c._key,
                Nome: CONCAT(c.FirstName, " ", c.LastName),
                Email: c.Email,
                Gênero: c.Gender
            }}
        """
    elif intent == "loyalty_points" and 'points' in entities:
        points = entities['points']
        return f"""
        FOR c IN Customers
            FILTER c.LoyaltyPoints > {points}
            RETURN {{
                CustomerID: c._key,
                Nome: CONCAT(c.FirstName, " ", c.LastName),
                Email: c.Email,
                Pontos: c.LoyaltyPoints
            }}
        """
    elif intent == "registration_date" and 'date' in entities:
        date = entities['date']
        return f"""
        FOR c IN Customers
            FILTER c.RegistrationDate > "{date}"
            RETURN {{
                CustomerID: c._key,
                Nome: CONCAT(c.FirstName, " ", c.LastName),
                Email: c.Email,
                DataCadastro: c.RegistrationDate
            }}
        """
    else:
        return None

# Função principal
def answer_question(question):
    intent, entities = extract_intent_entities(question)
    aql = build_aql(intent, entities)

    if not aql:
        return "Não consegui identificar a pergunta. Tente algo como 'Quais clientes moram na cidade de X?' ou 'Quais clientes preferem a loja Y?'"

    try:
        cursor = db.aql.execute(aql)
        results = list(cursor)

        if not results:
            return "Não encontrei clientes correspondentes."

        # Resumo dos dados
        summary = "Aqui estão alguns clientes:\n"
        for r in results[:10]:
            summary += f"{r['Nome']}\n"
        return summary

    except Exception as e:
        return f"Erro ao executar AQL: {str(e)}"


In [70]:
# Exemplo de perguntas
pergunta = "Quem prefere a loja Perez Ltd?"

# Executando as perguntas
print(answer_question(pergunta))



Aqui estão alguns clientes:
Janet Chavez

